08/06/2025- Food Waste Management 

Problem Statement

Food wastage is a significant issue, with many households and restaurants discarding surplus food while numerous people struggle with food insecurity. This project aims to develop a Local Food Wastage Management System, where:

1.Restaurants and individuals can list surplus food.

2.NGOs or individuals in need can claim the food.

3.SQL stores available food details and locations.

4.A Streamlit app enables interaction, filtering, CRUD operation and visualization. 


Business Use Cases

1.Connecting surplus food providers to those in need through a structured platform.

2.Reducing food waste by redistributing excess food efficiently.

3.Enhancing accessibility via geolocation features to locate food easily.

4.Data analysis on food wastage trends for better decision-making.

Step 1 : Download data into pandas

                i. For food providers

In [3]:
import pandas as pd
df_providers=pd.read_csv(r"D:\GUVI2\Data\providers_data.csv")

                 a. Cleaning the data

In [6]:
df_providers.dropna(axis=1, how="all", inplace=True)
#Not including unique rovider id here
df_providers['Name'].fillna('Unknown', inplace=True)
df_providers['Type'].fillna('Unknown',inplace=True)
df_providers['Address'].fillna('Unknown',inplace=True)
df_providers['City'].fillna('Unknown',inplace=True)
df_providers['Contact'].fillna('Unknown',inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_5940\1279989716.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Dell\AppData\Local\Temp\ipykernel_5940\1279989716.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

                b. Create a Table

In [7]:
import mysql.connector
connection=mysql.connector.connect(
    host="localhost",
    user="root",
    password="Sadguro161092",
    database="DSB42"

)
cursor=connection.cursor()

In [9]:
query="""
CREATE TABLE IF NOT EXISTS providers(
    Provider_ID INT,
    Name VARCHAR(100),
    Type VARCHAR(100),
    Address VARCHAR(100),
    City VARCHAR(100),
    Contact VARCHAR(50)

);
"""
cursor.execute(query)
connection.commit()

In [10]:
columns_needed=[
    'Provider_ID','Name','Type','Address','City','Contact'
]
dfproviders_selected=df_providers[columns_needed]
rows_to_tuples=[tuple(x) for x in dfproviders_selected.to_numpy()]

In [11]:
insert_data="""
 INSERT INTO providers(
Provider_ID,Name,Type,Address,City,Contact)
 VALUES(%s, %s, %s,%s,%s,%s)
 """

cursor.executemany(insert_data, rows_to_tuples)

                ii.For Reciver's dataset

In [12]:
df_receivers=pd.read_csv(r'D:\GUVI2\Data\receivers_data.csv')

                a.Clean the data

In [13]:
df_receivers.dropna(axis=1, how='all', inplace=True)
#Not including unique receiver id
df_receivers['Name'].fillna('Unknown', inplace=True)
df_receivers['Type'].fillna('Unknown', inplace=True)
df_receivers['City'].fillna('Unknown', inplace=True)
df_receivers['Contact'].fillna('Unknown',inplace=True)



C:\Users\Dell\AppData\Local\Temp\ipykernel_5940\911705583.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Dell\AppData\Local\Temp\ipykernel_5940\911705583.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

                b.Create a table

In [14]:
query="""
CREATE TABLE IF NOT EXISTS receivers(
    Receiver_ID INT,
    Name VARCHAR(100),
    Type VARCHAR(100),
    City VARCHAR(100),
    Contact VARCHAR(50)

);
"""
cursor.execute(query)
connection.commit()

In [15]:
columns_needed=[
    'Receiver_ID','Name','Type','City','Contact'
]
dfreceivers_selected=df_receivers[columns_needed]
rows_to_tuples=[tuple(x) for x in dfreceivers_selected.to_numpy()]

In [16]:
insert_data="""
 INSERT INTO receivers(
Receiver_ID,Name,Type,City,Contact)
 VALUES(%s, %s, %s,%s,%s)
 """

cursor.executemany(insert_data, rows_to_tuples)

        iii. For Food Listings Dataset

In [17]:
df_foodlistings=pd.read_csv(r"D:\GUVI2\Data\food_listings_data.csv")

                a.Clean data

In [18]:
# Drop columns with all NaN values (excluding 'Food_ID')
df_foodlistings.dropna(axis=1, how='all', inplace=True)

# Fill NaN in 'Food_Name' with 'Unknown'
df_foodlistings['Food_Name'].fillna('Unknown', inplace=True)

# Fill NaN in 'Quantity' with the mean of the column (only if it's numeric)
df_foodlistings['Quantity'] = df_foodlistings['Quantity'].fillna(df_foodlistings['Quantity'].mean())

# Drop rows where 'Expiry_Date' or 'Provider_ID' are NaN
df_foodlistings.dropna(subset=['Expiry_Date', 'Provider_ID'], inplace=True)

# Fill NaN values for categorical columns in one go
df_foodlistings.fillna({
    'Provider_Type': 'Unknown',
    'Location': 'Unknown',
    'Food_Type': 'Unknown',
    'Meal_Type': 'Unknown'
}, inplace=True)

# Alternatively, you can use a more general approach if you'd like to fill any NaN
# values in object (string) columns with a default value, and numeric columns with the mean.
df_foodlistings = df_foodlistings.apply(lambda x: x.fillna(x.mean()) if x.dtype in ['float64', 'int64'] else x.fillna('Unknown'))


C:\Users\Dell\AppData\Local\Temp\ipykernel_5940\3907010888.py:5: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [19]:
query="""
CREATE TABLE IF NOT EXISTS foodlistings (
    Food_ID INT,                   -- Unique identifier for each food item
    Food_Name VARCHAR(100),        -- Name of the food item
    Quantity INT,                  -- Quantity available for distribution
    Expiry_Date DATE,              -- Expiry date of the food item
    Provider_ID INT,               -- Reference to the provider offering the food
    Provider_Type VARCHAR(100),    -- Type of provider offering the food
    Location VARCHAR(100),         -- City where the food is available
    Food_Type VARCHAR(100),        -- Category of food (e.g., Vegetarian, Non-Vegetarian, Vegan)
    Meal_Type VARCHAR(100)         -- Type of meal (e.g., Breakfast, Lunch, Dinner, Snacks)


);
"""
cursor.execute(query)
connection.commit()

In [20]:
columns_needed=[
    'Food_ID','Food_Name','Quantity','Expiry_Date','Provider_ID','Provider_Type','Location','Food_Type','Meal_Type'
]
df_foodlistings_selected=df_foodlistings[columns_needed]
rows_to_tuples=[tuple(x) for x in df_foodlistings_selected.to_numpy()]

In [21]:
insert_data = """
INSERT INTO foodlistings (
    Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, 
    Provider_Type, Location, Food_Type, Meal_Type
) 
VALUES (%s, %s, %s, STR_TO_DATE(%s, '%c/%d/%Y'), %s, %s, %s, %s, %s)
"""


cursor.executemany(insert_data, rows_to_tuples)

        iv. For Claims Dataset

In [22]:
df_claims=pd.read_csv(r'D:\GUVI2\Data\claims_data.csv')

                a. Clean Data

In [23]:
df_claims.head()

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17


In [24]:
df_claims.dropna(subset=['Claim_ID'], inplace=True)
df_claims.dropna(subset=['Food_ID'],inplace=True)
df_claims.dropna(subset=['Receiver_ID'],inplace=True)
df_claims['Status'].fillna('Unknown', inplace=True)
df_claims['Timestamp'].fillna('Unknown', inplace=True)


C:\Users\Dell\AppData\Local\Temp\ipykernel_5940\3305683245.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Dell\AppData\Local\Temp\ipykernel_5940\3305683245.py:5: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [25]:
query="""
CREATE TABLE IF NOT EXISTS claims (
    Claim_ID INT,
    Food_ID INT,
    Receiver_ID INT,
    Status VARCHAR(100),
    Timestamp VARCHAR(100)

);
"""
cursor.execute(query)
connection.commit()

In [26]:
columns_needed=[
    'Claim_ID','Food_ID','Receiver_ID','Status','Timestamp'
]
df_claims_selected=df_claims[columns_needed]
rows_to_tuples=[tuple(x) for x in df_claims_selected.to_numpy()]

In [27]:
insert_data = """
INSERT INTO claims (
    Claim_ID, Food_ID, Receiver_ID, Status, Timestamp
) 
VALUES (%s, %s, %s, %s, %s)
"""


cursor.executemany(insert_data, rows_to_tuples)

SOME BASIC EDA OPERATIONS:

In [28]:
print("\nDuplicate Rows in Providers Dataset:",df_providers.duplicated().sum())
print("\nDuplicate Rows in Receivers Dataset:",df_receivers.duplicated().sum())
print("\nDuplicate Rows in Food Listings Dataset:",df_foodlistings.duplicated().sum())
print("\nDuplicate Rows in Claims Dataset:", df_claims.duplicated().sum())



Duplicate Rows in Providers Dataset: 0

Duplicate Rows in Receivers Dataset: 0

Duplicate Rows in Food Listings Dataset: 0

Duplicate Rows in Claims Dataset: 0


In [34]:
print("Shape:",df_providers.shape)
print("Data Types:\n",df_providers.dtypes)
print("Missing values: \n",df_providers.isnull().sum())
print("Sample Data:\n", df_providers.head(),"\n")

Shape: (1000, 6)
Data Types:
 Provider_ID     int64
Name           object
Type           object
Address        object
City           object
Contact        object
dtype: object
Missing values: 
 Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64
Sample Data:
    Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince and Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell T

In [35]:
print("Shape:",df_receivers.shape)
print("Data Types:\n",df_receivers.dtypes)
print("Missing values: \n",df_receivers.isnull().sum())
print("Sample Data:\n", df_receivers.head(),"\n")

Shape: (1000, 5)
Data Types:
 Receiver_ID     int64
Name           object
Type           object
City           object
Contact        object
dtype: object
Missing values: 
 Receiver_ID    0
Name           0
Type           0
City           0
Contact        0
dtype: int64
Sample Data:
    Receiver_ID          Name        Type               City           Contact
0            1  Donald Gomez     Shelter     Port Carlburgh     (955)922-5295
1            2  Laurie Ramos  Individual         Lewisburgh      761.042.1570
2            3  Ashley Mckee         NGO  South Randalltown  691-023-0094x856
3            4    Erika Rose         NGO   South Shaneville        8296491111
4            5   John Romero  Individual          Bakerport      067.491.0154 



In [36]:
print("Shape:",df_foodlistings.shape)
print("Data Types:\n",df_foodlistings.dtypes)
print("Missing values: \n",df_foodlistings.isnull().sum())
print("Sample Data:\n", df_foodlistings.head(),"\n")

Shape: (1000, 9)
Data Types:
 Food_ID           int64
Food_Name        object
Quantity          int64
Expiry_Date      object
Provider_ID       int64
Provider_Type    object
Location         object
Food_Type        object
Meal_Type        object
dtype: object
Missing values: 
 Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64
Sample Data:
    Food_ID Food_Name  Quantity Expiry_Date  Provider_ID     Provider_Type  \
0        1     Bread        43   3/17/2025          110     Grocery Store   
1        2      Soup        22   3/24/2025          791     Grocery Store   
2        3    Fruits        46   3/28/2025          478  Catering Service   
3        4    Fruits        15   3/16/2025          930        Restaurant   
4        5      Soup        14   3/19/2025          279        Restaurant   

           Location       Food_Type  Meal_Type  
0  South Kel

In [37]:
print("Shape:",df_claims.shape)
print("Data Types:\n",df_claims.dtypes)
print("Missing values: \n",df_claims.isnull().sum())
print("Sample Data:\n", df_claims.head(),"\n")

Shape: (1000, 5)
Data Types:
 Claim_ID        int64
Food_ID         int64
Receiver_ID     int64
Status         object
Timestamp      object
dtype: object
Missing values: 
 Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64
Sample Data:
    Claim_ID  Food_ID  Receiver_ID     Status        Timestamp
0         1      164          908    Pending    3/5/2025 5:26
1         2      353          391  Cancelled  3/11/2025 10:24
2         3      626          492  Completed   3/21/2025 0:59
3         4       61          933  Cancelled    3/4/2025 9:08
4         5      345          229    Pending  3/14/2025 15:17 



SQL QUERIES

        A. FOOD PROVIDERS AND RECEIVERS

             1.How many food providers and receivers are there in each city?

In [ ]:
query="""
SELECT
    p.City,
    COUNT(DISTINCT p.Provider_ID) AS Num_Providers,
    COUNT(DISTINCT r.Receiver_ID) AS Num_Receivers
FROM providers p
LEFT JOIN receivers r ON p.City=r.City
GROUP BY p.City


"""

cursor.execute(query)
results=cursor.fetchall()
for City, Num_Providers, Num_Receivers in results:
    print(f"{City}: {Num_Providers} Providers  |  {Num_Receivers} Receivers")

Adambury: 1 Providers  |  0 Receivers
Adamsview: 1 Providers  |  0 Receivers
Adamsville: 1 Providers  |  0 Receivers
Aguirreville: 1 Providers  |  0 Receivers
Alexanderchester: 1 Providers  |  0 Receivers
Alexanderstad: 1 Providers  |  0 Receivers
Allenborough: 1 Providers  |  1 Receivers
Allenton: 1 Providers  |  0 Receivers
Amandaborough: 1 Providers  |  0 Receivers
Amandashire: 1 Providers  |  0 Receivers
Amberton: 1 Providers  |  0 Receivers
Ambertown: 1 Providers  |  0 Receivers
Amyport: 1 Providers  |  0 Receivers
Andersonmouth: 1 Providers  |  0 Receivers
Andersonville: 1 Providers  |  0 Receivers
Andreaborough: 1 Providers  |  0 Receivers
Andrewsmouth: 1 Providers  |  0 Receivers
Andrewstad: 1 Providers  |  0 Receivers
Anitashire: 1 Providers  |  0 Receivers
Annahaven: 1 Providers  |  0 Receivers
Annetteburgh: 1 Providers  |  0 Receivers
Anneville: 1 Providers  |  0 Receivers
Anthonyborough: 1 Providers  |  0 Receivers
Anthonychester: 1 Providers  |  0 Receivers
Anthonyfort: 1 

                 2.Which type of food provider (restaurant, grocery store, etc.) contributes the most food

In [ ]:
query=""" 
SELECT 
    p.Type,
    SUM(f.Quantity) AS Food_Total
FROM foodlistings f
JOIN providers p on f.Provider_ID=p.Provider_ID
GROUP BY p.Type
ORDER BY Food_Total DESC
LIMIT 1;
"""


    
cursor.execute(query)
result = cursor.fetchone()  # Only one row expected due to LIMIT 1

# Unpack and print the result
if result:
    type_, total_food = result
    print(f"Top Provider Type: {type_} | Total Food Quantity: {total_food}")
else:
    print("No data found.")

Top Provider Type: Restaurant | Total Food Quantity: 533071


        3. What is the contact information of food providers in a specific city?

In [ ]:
city_name=input("Enter city name to get contact information of food providers in that city: ")
query=""" 
SELECT DISTINCT Name,Contact
FROM providers p
WHERE City=%s;

"""
cursor.execute(query, (city_name,))
results = cursor.fetchall()
for name, contact in results:
    print(f"Name: {name} | Contact: {contact}")


Name: Clark, Prince and Williams | Contact: 556.944.8935x401


                4.Which receivers have claimed the most food?

In [ ]:
query=""" 
SELECT r.Name, SUM(f.Quantity) AS Food_Claimed
FROM claims c
JOIN receivers r ON c.Receiver_ID =r.receiver_ID
JOIN foodlistings f ON c.Food_ID=f.Food_ID
WHERE c.Status='Completed' 
GROUP BY r.Name
ORDER BY Food_Claimed DESC
LIMIT 1;


"""
cursor.execute(query)
result = cursor.fetchone() 

if result:
    name_, food_claimed = result
    print(f"Top Receiver: {name_} | Total Food Quantity: {food_claimed}")
else:
    print("No data found.")

Top Receiver: Derek Potter | Total Food Quantity: 13860


    B. FOOD LSITINGS AND AVAILABILITY

        5.What is the total quantity of food available from all providers?

In [ ]:
query="""
SELECT SUM(f.quantity) as Total_Food
FROM foodlistings f

 """

cursor.execute(query)
result = cursor.fetchone() 

if result:
    Total_Food = result
    print(f" Total Food Quantity: {Total_Food}")
else:
    print("No data found.")

 Total Food Quantity: (Decimal('180558'),)


        6.Which city has the highest number of food listings?

In [ ]:
query=''' 
SELECT Location AS City, COUNT(*) AS Max_Listings
FROM foodlistings
GROUP BY Location
ORDER BY Max_Listings
LIMIT 1;

'''
cursor.execute(query)
result = cursor.fetchone() 

if result:
    Max_Listings = result
    print(f" City with Highest Number of Food listings: {Max_Listings}")
else:
    print("No data found.")


 City with Highest Number of Food listings: ('South Kellyville', 7)


            7. What are the most commonly available food types?

In [ ]:
query=''' 
SELECT f.Food_Type, COUNT(*) AS Common_Type
FROM foodlistings f
GROUP BY f.Food_Type
ORDER BY Common_Type
LIMIT 1;
'''
cursor.execute(query)
result = cursor.fetchone() 

if result:
    Common_Type= result
    print(f" Commonly Available Food Types: {Common_Type}")
else:
    print("No data found.")

 Commonly Available Food Types: ('Non-Vegetarian', 2310)


CLAIMS AND DISTRIBUTION

            8.How many food claims have been made for each food item?

In [ ]:
query='''   
SELECT c.Food_ID, COUNT(*) AS Num_Claims
FROM claims c
GROUP BY Food_ID
ORDER BY Num_claims;

'''
cursor.execute(query)
results = cursor.fetchall()
for Food_ID, Num_Claims in results:
    print(f"Food ID: {Food_ID} | Claims: {Num_Claims}")

Food ID: 61 | Claims: 2
Food ID: 273 | Claims: 2
Food ID: 735 | Claims: 2
Food ID: 382 | Claims: 2
Food ID: 278 | Claims: 2
Food ID: 669 | Claims: 2
Food ID: 694 | Claims: 2
Food ID: 426 | Claims: 2
Food ID: 792 | Claims: 2
Food ID: 281 | Claims: 2
Food ID: 959 | Claims: 2
Food ID: 586 | Claims: 2
Food ID: 193 | Claims: 2
Food ID: 905 | Claims: 2
Food ID: 713 | Claims: 2
Food ID: 597 | Claims: 2
Food ID: 757 | Claims: 2
Food ID: 825 | Claims: 2
Food ID: 556 | Claims: 2
Food ID: 746 | Claims: 2
Food ID: 895 | Claims: 2
Food ID: 543 | Claims: 2
Food ID: 990 | Claims: 2
Food ID: 119 | Claims: 2
Food ID: 424 | Claims: 2
Food ID: 310 | Claims: 2
Food ID: 526 | Claims: 2
Food ID: 708 | Claims: 2
Food ID: 641 | Claims: 2
Food ID: 805 | Claims: 2
Food ID: 197 | Claims: 2
Food ID: 790 | Claims: 2
Food ID: 829 | Claims: 2
Food ID: 378 | Claims: 2
Food ID: 136 | Claims: 2
Food ID: 741 | Claims: 2
Food ID: 469 | Claims: 2
Food ID: 162 | Claims: 2
Food ID: 864 | Claims: 2
Food ID: 108 | Claims: 2
F

         9. Which provider has had the highest number of successful food claims?
     

In [ ]:
query=''' 
SELECT p.Provider_ID, COUNT(*) AS Succesful_Claims
FROM claims c
JOIN foodlistings f ON c.Food_ID=f.Food_ID
JOIN providers p ON p.Provider_ID= f.Provider_ID
WHERE c.Status='Completed'
GROUP BY p.Provider_ID
ORDER BY Succesful_Claims DESC
LIMIT 1;

'''
cursor.execute(query)
result = cursor.fetchone() 
if result:
    providerID_, successful_claims = result
    print(f"Provider with the highest food claims: {providerID_} | Total Successful Claims: {successful_claims}")
else:
    print("No data found.")

Provider with the highest food claims: 709 | Total Successful Claims: 770


                10. What percentage of food claims are completed vs. pending vs. canceled?

In [ ]:
query=''' 
SELECT 
        Status,
        COUNT(*) AS count,
        ROUND(100.0*COUNT(*)/(SELECT COUNT(*) FROM claims),2) AS Percentage
FROM claims


GROUP BY Status

ORDER BY 
        Percentage DESC;

'''

cursor.execute(query)
results = cursor.fetchall()

print(f"{'Status':<12} | {'Claims':<6} | {'Percentage':<10}")
print("-" * 35)

for status, count, percentage in results:
    print(f"{status:<12} | {count:<6} | {percentage:<10.2f}%")


Status       | Claims | Percentage
-----------------------------------
Completed    | 678    | 33.90     %
Cancelled    | 672    | 33.60     %
Pending      | 650    | 32.50     %


        ANALYSIS AND INSIGHTS   

                       11. What is the average quantity of food claimed per receiver?

In [ ]:
query=''' 
SELECT r.Name, AVG(f.Quantity) AS Avg_Claimed
FROM claims c
JOIN receivers r ON c.receiver_ID=r.receiver_ID
JOIN foodlistings f ON c.Food_ID=f.Food_ID
GROUP BY r.Name

'''

cursor.execute(query)
results = cursor.fetchall()

print(f"{'Name':<30} | {'Quantity':<6}")
print("-" * 100)

for name_, Avg_Claimed in results:
    print(f"{name_:<30} | {Avg_Claimed:<6}")



Name                           | Quantity
----------------------------------------------------------------------------------------------------
Donald Gomez                   | 35.5000
Laurie Ramos                   | 47.0000
Ashley Mckee                   | 26.0000
Erika Rose                     | 30.5000
Mandy Sutton PhD               | 15.0000
Kenneth Baker                  | 31.0000
Emily Turner                   | 34.0000
Mary Salazar                   | 26.5000
Jennifer Bishop                | 33.0000
Brian Fischer                  | 28.0000
Donald Kelly                   | 44.0000
Elizabeth Lewis                | 4.0000
Kimberly Lindsey               | 14.0000
Johnny Page                    | 34.0000
Mary Franklin                  | 29.0000
Rose Powell                    | 26.0000
Audrey Perez                   | 45.0000
Anita Kerr                     | 26.0000
Megan Alvarez                  | 13.0000
James Castaneda                | 14.0000
Angel Adams                    | 49.00

          12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?


In [ ]:
query=''' 
SELECT Meal_Type, COUNT(*) AS Num_claims
FROM foodlistings f
JOIN claims c ON f.Food_ID=c.Food_ID
GROUP BY Meal_type
ORDER BY Num_claims DESC
LIMIT 1;

'''
cursor.execute(query)
results=cursor.fetchone()
meal_type_, Num_claims=results
print(f"Most Claimed Type of Meal: {meal_type_}\nQuantity: {Num_claims}")

Most Claimed Type of Meal: Breakfast
Quantity: 3892


                13.What is the total quantity of food donated by each provider?

In [ ]:
query=''' 
SELECT p.Provider_ID, COUNT(*) AS Num_Donated
FROM providers p
JOIN foodlistings f ON p.Provider_ID=f.Provider_ID
GROUP BY Provider_ID
ORDER BY Num_Donated;
'''
cursor.execute(query)
results=cursor.fetchall()
print(f"{'ID':<30} | {'Quantity':<6}")
print("-" * 70)

for providerID__, Num_Donated in results:
    print(f"{providerID_:<30} | {Num_Donated:<6}")

ID                             | Quantity
----------------------------------------------------------------------
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709                            | 77    
709    

                14. Average number of claims per receiver

In [ ]:
query=''' 
SELECT Receiver_ID, COUNT(*) AS total_claims
FROM claims
GROUP BY Receiver_ID

'''
cursor.execute(query)
results=cursor.fetchall()
for total_claims, recevierID_ in results:
    print(f"{recevierID_}   |  {total_claims}")

6   |  908
4   |  391
4   |  492
4   |  933
2   |  229
4   |  607
4   |  256
4   |  900
2   |  807
2   |  975
6   |  118
2   |  739
4   |  959
6   |  233
4   |  97
2   |  836
2   |  588
4   |  729
4   |  399
2   |  88
8   |  493
2   |  757
2   |  526
6   |  717
4   |  10
2   |  487
4   |  189
4   |  769
6   |  228
2   |  546
10   |  371
4   |  386
2   |  271
6   |  93
2   |  562
2   |  920
2   |  380
6   |  923
4   |  789
2   |  336
6   |  306
6   |  31
2   |  569
4   |  412
4   |  48
4   |  373
4   |  678
4   |  804
6   |  255
4   |  517
2   |  449
6   |  737
4   |  601
2   |  452
2   |  295
4   |  149
6   |  550
2   |  279
8   |  90
6   |  532
4   |  730
4   |  817
4   |  321
6   |  45
4   |  479
4   |  897
2   |  44
2   |  222
4   |  62
2   |  161
6   |  185
8   |  324
2   |  151
4   |  795
4   |  35
4   |  876
2   |  638
6   |  695
6   |  554
2   |  624
4   |  425
2   |  918
2   |  310
2   |  632
6   |  27
4   |  65
2   |  129
2   |  523
2   |  352
6   |  500
2   |  802
2   |  116


                15. Providers with most diverse food offerings

In [ ]:
query=''' 
SELECT 
        p.Name AS Pname,
        COUNT(DISTINCT f.Food_type) AS Food_Type_Count
FROM foodlistings f
JOIN providers p ON p.Provider_ID =f.Provider_ID
GROUP BY p.Name
ORDER BY Food_Type_Count DESC
LIMIT 1;
'''

cursor.execute(query)
results=cursor.fetchall()

for Provider_ID, Food_Type_Count in results:
    print(f"{Provider_ID}    {Food_Type_Count}  ")

Anderson, Nicholson and Bruce    3  


                16.Top 5 active Food providers based on 'Completed' Status of Claims

In [ ]:
query=''' 
SELECT 
    p.Provider_ID,
    p.Name as Provider_name,
    COUNT(c.Claim_ID)  AS Claim_num
FROM
    providers p

JOIN foodlistings f ON p.Provider_ID=f.Provider_ID
JOIN claims c ON f.Food_ID=c.Food_ID
WHERE c.Status='Completed'
GROUP BY 
    p.Provider_ID,p.Name
ORDER BY 
    Claim_num DESC
LIMIT 5; 

'''


cursor.execute(query)
results=cursor.fetchall()

for Provider_ID_, Name_, Claim_num in results:
    print(f"{Provider_ID_}   {Name_}    {Claim_num}  ")


709   Barry Group    770  
241   Harper, Blake and Alexander    616  
752   Butler-Richardson    616  
596   Barnes, Castro and Curtis    616  
967   Miller Inc    616  


                17. Distribution of Claims by Status

In [ ]:
query=''' 
SELECT 
        Status,
        COUNT(Claim_ID) AS Claim_Count,
        ROUND(COUNT(Claim_ID)/(SELECT COUNT(*) FROM claims)*100,2) AS Claim_Percentage
FROM claims 
GROUP BY Status
'''
cursor.execute(query)
results=cursor.fetchall()

for Claim_ID_, Claim_Count, Claim_Percentage in results:
    print(f"{Claim_ID_}     {Claim_Count}    {Claim_Percentage}  ")


Pending     650    32.50  
Cancelled     672    33.60  
Completed     678    33.90  


                18. Providers with no claims

In [ ]:
query=''' 
SELECT 
    p.Provider_ID,
    p.Name AS Provider_Name
FROM providers p
WHERE NOT EXISTS (
        SELECT 1
        FROM foodlistings f
        JOIN claims c ON f.Food_ID=c.Food_ID
        WHERE f.Provider_ID=p.provider_ID

);

'''
cursor.execute(query)
results=cursor.fetchall()

print("Providers with No Food Claims:")
print("-" * 40)
for provider_id, provider_name in results:
    print(f"Provider ID: {provider_id:<10} | Provider Name: {provider_name}")

Providers with No Food Claims:
----------------------------------------
Provider ID: 3          | Provider Name: Miller-Black
Provider ID: 5          | Provider Name: Coleman-Farley
Provider ID: 6          | Provider Name: Lawson-Walters
Provider ID: 8          | Provider Name: Curtis-Lewis
Provider ID: 10         | Provider Name: Hall, Nguyen and Martinez
Provider ID: 11         | Provider Name: Flores, Jackson and Ford
Provider ID: 12         | Provider Name: Miller Ltd
Provider ID: 13         | Provider Name: Parker, Flores and Jennings
Provider ID: 14         | Provider Name: Nguyen-Tran
Provider ID: 15         | Provider Name: Valdez-Diaz
Provider ID: 18         | Provider Name: Edwards-Turner
Provider ID: 19         | Provider Name: Davis Ltd
Provider ID: 21         | Provider Name: Cross Group
Provider ID: 23         | Provider Name: Johnson Ltd
Provider ID: 25         | Provider Name: Gutierrez-Palmer
Provider ID: 27         | Provider Name: Lester Inc
Provider ID: 28         |

In [ ]:
query = '''
SELECT 
    p.Provider_ID,
    p.Name AS Provider_Name
FROM 
    providers p
LEFT JOIN 
    foodlistings f ON p.Provider_ID = f.Provider_ID
LEFT JOIN 
    claims c ON f.Food_ID = c.Food_ID
WHERE 
    c.Claim_ID IS NULL;
'''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Print results
print("Providers with No Food Claims:")
print("-" * 40)
for provider_id, provider_name in results:
    print(f"Provider ID: {provider_id:<10} | Provider Name: {provider_name}")


Providers with No Food Claims:
----------------------------------------
Provider ID: 2          | Provider Name: Nielsen, Johnson and Fuller
Provider ID: 2          | Provider Name: Nielsen, Johnson and Fuller
Provider ID: 2          | Provider Name: Nielsen, Johnson and Fuller
Provider ID: 2          | Provider Name: Nielsen, Johnson and Fuller
Provider ID: 2          | Provider Name: Nielsen, Johnson and Fuller
Provider ID: 2          | Provider Name: Nielsen, Johnson and Fuller
Provider ID: 2          | Provider Name: Nielsen, Johnson and Fuller
Provider ID: 3          | Provider Name: Miller-Black
Provider ID: 5          | Provider Name: Coleman-Farley
Provider ID: 5          | Provider Name: Coleman-Farley
Provider ID: 5          | Provider Name: Coleman-Farley
Provider ID: 5          | Provider Name: Coleman-Farley
Provider ID: 5          | Provider Name: Coleman-Farley
Provider ID: 5          | Provider Name: Coleman-Farley
Provider ID: 5          | Provider Name: Coleman-Farley

                19. Total claims and quantity per Meal type

In [ ]:
query=''' 
SELECT 
        f.Meal_Type,
        COUNT(c.Claim_ID) AS Total_Claims,
        SUM(f.Quantity) AS Total_Quantity
FROM
    claims c
JOIN foodlistings f ON c.Food_ID=f.Food_ID
GROUP BY f.Meal_Type, f.Quantity
ORDER BY Total_Quantity



'''
cursor.execute(query)

results = cursor.fetchall()

print("Total claims and quantity per meal type:")
print("-" * 40)
for meal_type_, Total_Claims, Total_Quantity in results:
    print(f"Meal Type: {meal_type_:<10} | Quantity: {Total_Quantity:<12}    |  Claims: {Total_Claims}")


Total claims and quantity per meal type:
----------------------------------------
Meal Type: Snacks     | Quantity: 28              |  Claims: 28
Meal Type: Lunch      | Quantity: 28              |  Claims: 28
Meal Type: Dinner     | Quantity: 56              |  Claims: 56
Meal Type: Lunch      | Quantity: 84              |  Claims: 42
Meal Type: Snacks     | Quantity: 112             |  Claims: 14
Meal Type: Snacks     | Quantity: 126             |  Claims: 42
Meal Type: Snacks     | Quantity: 140             |  Claims: 70
Meal Type: Dinner     | Quantity: 140             |  Claims: 70
Meal Type: Snacks     | Quantity: 140             |  Claims: 28
Meal Type: Snacks     | Quantity: 168             |  Claims: 42
Meal Type: Lunch      | Quantity: 168             |  Claims: 42
Meal Type: Snacks     | Quantity: 196             |  Claims: 14
Meal Type: Breakfast  | Quantity: 196             |  Claims: 98
Meal Type: Breakfast  | Quantity: 210             |  Claims: 70
Meal Type: Snacks     

           20. Top 10 Common Provider Locations for Food     

In [ ]:
query=''' 
SELECT 
    p.City AS Provider_city,
    COUNT(*) AS Provider_by_city
FROM providers p
GROUP BY p.City
ORDER BY Provider_by_city
LIMIT 10

'''

cursor.execute(query)

results = cursor.fetchall()

print("Top 10 most common provider location for food:")
print("-" * 40)
for Provider_city, Provider_by_city in results:
    print(f"Provider: {Provider_city:<30} | City: {Provider_by_city:<12}  ")

Top 10 most common provider location for food:
----------------------------------------
Provider: New Jessica                    | City: 11            
Provider: East Sheena                    | City: 11            
Provider: Lake Jesusview                 | City: 11            
Provider: Mendezmouth                    | City: 11            
Provider: Valentineside                  | City: 11            
Provider: Shannonside                    | City: 11            
Provider: Lake Christopherburgh          | City: 11            
Provider: Washingtonville                | City: 11            
Provider: Tinamouth                      | City: 11            
Provider: West Cherylfort                | City: 11            


                21. Most Common Type of Receiver

In [ ]:
query=''' 
SELECT 
    r.Type as Receiver_type,
    COUNT(*) AS Receiver_type_count
FROM receivers r
GROUP BY r.Type
ORDER BY Receiver_type_count DESC
LIMIT 1;
'''
cursor.execute(query)

results = cursor.fetchall()

print("Most Common Type of Receiver")
print("-" * 40)
for Receiver_type, Receiver_type_count in results:
    print(f"Receiver Type: {Receiver_type:<10} | Count: {Receiver_type_count:<12}  ")

Most Common Type of Receiver
----------------------------------------
Receiver Type: NGO        | Count: 2740          


                22. Receiver type by count

In [ ]:
query=''' 
SELECT 
    r.Type AS Receiver_type,
    COUNT(*) AS Receiver_Type_Count
FROM receivers r
GROUP BY r.Type
ORDER BY Receiver_Type_Count;


'''
cursor.execute(query)

results = cursor.fetchall()

print("Receiver type of count")
print("-" * 40)
for Receiver_type, Receiver_Type_Count in results:
    print(f"Receiver Type: {Receiver_type:<10} | Count: {Receiver_Type_Count:<12}  ")

Receiver type of count
----------------------------------------
Receiver Type: Individual | Count: 2170          
Receiver Type: Shelter    | Count: 2460          
Receiver Type: Charity    | Count: 2630          
Receiver Type: NGO        | Count: 2740          


                23. Receiver count by city

In [ ]:
query= ''' 
SELECT  
    City AS Receiver_City,
    COUNT(*) AS Receiver_city_count
FROM receivers
GROUP BY City
ORDER BY Receiver_city_count


'''

cursor.execute(query)

results = cursor.fetchall()

print("Receiver cities by Count")
print("-" * 40)
for Receiver_city, Receiver_city_count in results:
    print(f"Receiver Type: {Receiver_city:<30} | Count: {Receiver_city_count:<12}  ")

Receiver cities by Count
----------------------------------------
Receiver Type: Port Carlburgh                 | Count: 10            
Receiver Type: Lewisburgh                     | Count: 10            
Receiver Type: South Randalltown              | Count: 10            
Receiver Type: South Shaneville               | Count: 10            
Receiver Type: Bakerport                      | Count: 10            
Receiver Type: East Sharimouth                | Count: 10            
Receiver Type: South Edwinborough             | Count: 10            
Receiver Type: Benjaminburgh                  | Count: 10            
Receiver Type: West Robert                    | Count: 10            
Receiver Type: Wrightland                     | Count: 10            
Receiver Type: South Gregorymouth             | Count: 10            
Receiver Type: Katherineside                  | Count: 10            
Receiver Type: Lake Matthewstad               | Count: 10            
Receiver Type: Burkeside

                24. Providers with the most expired food items

In [ ]:
query=''' 
SELECT 
    p.Name AS Provider_Name,
    COUNT(f.Food_ID) AS Expired_food_count
FROM foodlistings f
JOIN providers p ON f.Provider_ID=p.Provider_ID
WHERE f.Expiry_Date<CURDATE()
GROUP BY p.Provider_ID, p.Name
ORDER BY Expired_food_count DESC
LIMIT 5;

'''
cursor.execute(query)

results = cursor.fetchall()

print("Providers with the most expired food items:")
print("-" * 40)
for Provider_ID_, Expired_food_count in results:
    print(f"Provider: {Provider_ID_:<30} | Count: {Expired_food_count:<12}  ")

Providers with the most expired food items:
----------------------------------------
Provider: Barry Group                    | Count: 462           
Provider: Daniels-Kim                    | Count: 385           
Provider: Campbell LLC                   | Count: 385           
Provider: Smith Group                    | Count: 385           
Provider: Evans, Wright and Mitchell     | Count: 308           


In [ ]:
%%writefile food.py
import streamlit as st
import pandas as pd
import mysql.connector
import plotly.express as px
import seaborn as sns


# Database Connection
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Sadguro161092",
    database="DSB42"
)
cursor = connection.cursor()

# Streamlit Configuration
st.set_page_config(page_title="Local Food Waste Management Dashboard", layout="wide", page_icon="🍽️")
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ['Project Introduction', 'Data Overview', 'Visualizations', 'SQL Queries', 'Writer Info'])

if page == 'Project Introduction':
    st.title("🍽️ Local Food Waste Management System")

    # Project Overview
    st.markdown("### 🌐 Project Overview")
    st.markdown("""
    The **Local Food Waste Management System** is designed to support the **redistribution of surplus food**  
    by connecting **providers** (e.g., restaurants, grocery stores) with **receivers** (e.g., NGOs, shelters).  
    The aim is to **reduce food waste** and tackle **community hunger** using structured, data-driven processes.
    """)

    st.image("D:/GUVI2/Foodwastemanagemnt/localfwm.png", caption="Smart Food Redistribution", use_container_width=True)

    # Why This Matters
    st.markdown("### 📍 Why This Matters")
    st.markdown("""
    - Over **1.3 billion tons** of food is wasted globally each year  
    - Millions face **food insecurity** despite available surplus  
    - This system bridges the gap between **excess and need**
    """)

    # Key Features
    st.markdown("### ⚙️ Key Features")
    st.markdown("""
    - City-level tracking of providers and receivers  
    - Real-time listings with expiry visibility  
    - Dashboards to analyze donations and claims  
    - Shelf-life tracking to prevent spoilage  
    - Direct contact access for coordination  
    """)

    st.image("D:/GUVI2/Foodwastemanagemnt/localfwm2.png", caption="From Waste to Worth", use_container_width=True)

    st.markdown("---")

    # Project Objectives
    st.markdown("### 🎯 Project Objectives")
    st.markdown("""
    - Ensure **timely collection and delivery** of surplus food  
    - Use **data analytics** to improve distribution patterns  
    - Enable **transparency** through a SQL-integrated dashboard  
    """)

    # Technologies Used
    st.markdown("### 💻 Technologies Used")
    st.markdown("""
    - **Frontend**: Streamlit  
    - **Database**: MySQL  
    - **Backend**: Python  
    - **Visualization**: Matplotlib, Seaborn, Plotly  
    """)

    st.markdown("---")
    st.success("🔍 Explore the **'SQL Queries'** section to gain insights into food distribution patterns.")





elif page == 'Data Overview':
    st.title("📊 Data Overview")

    st.markdown("""
    Below are the datasets used in the **Local Food Waste Management** system.  
    Use the dropdown to explore the structure and contents of each database table.
    """)

    table = st.selectbox("📁 Select Table", ["providers", "receivers", "foodlistings", "claims"])
    df = pd.read_sql(f"SELECT * FROM {table}", connection)

    st.markdown(f"**📌 Table Selected:** `{table}`")
    st.markdown(f"**Rows:** {df.shape[0]}  **Columns:** {df.shape[1]}")

    

    st.markdown("### 🔍 Table Preview")
    st.dataframe(df, use_container_width=True)

#------------------------------------------------------------------------------------------------------------------
elif page=='Visualizations':
    providers = pd.read_sql("SELECT * FROM providers", connection)
    receivers = pd.read_sql("SELECT * FROM receivers", connection)
    foodlistings = pd.read_sql("SELECT * FROM foodlistings", connection)
    claims = pd.read_sql("SELECT * FROM claims", connection)
    st.title("📊 EDA: Food Sharing System")

    st.header("1. Dataset Overview")
    st.write("**Food Listings:**", foodlistings.shape)
    st.write("**Providers:**", providers.shape)
    st.write("**Receivers:**", receivers.shape)
    st.write("**Claims:**", claims.shape)

    # Show sample data
    with st.expander("Show sample data"):
        st.subheader("Food Listings Sample")
        st.dataframe(foodlistings.head())

        st.subheader("Providers Sample")
        st.dataframe(providers.head())

        st.subheader("Receivers Sample")
        st.dataframe(receivers.head())

        st.subheader("Claims Sample")
        st.dataframe(claims.head())

    # --- Food Listings EDA ---
    st.header("Food Listings Insights")

   

    # Food type breakdown
    if 'Food_Type' in foodlistings.columns:
        fig2 = px.pie(foodlistings, names='Food_Type', title='Food Type Distribution')
        st.plotly_chart(fig2)

    # Meal type
    if 'Meal_Type' in foodlistings.columns:
        meal_counts = foodlistings['Meal_Type'].value_counts().reset_index()
        meal_counts.columns = ['Meal_Type', 'Count']

        fig3 = px.bar(
            meal_counts,
            x='Meal_Type', y='Count',
            labels={'Meal_Type': 'Meal Type', 'Count': 'Count'},
            title="Meal Type Distribution"
        )


    



    # --- Receiver Analysis ---
    st.header(" Receiver Insights")

   
    if 'Type' in receivers.columns:
        fig8 = px.pie(receivers, names='Type', title='Receiver Type Breakdown')
        st.plotly_chart(fig8)

    # --- Claims Overview ---
    st.header("Claim Status Summary")

    if 'Status' in claims.columns:
        fig9 = px.pie(claims, names='Status', title='Claim Status Distribution')
        st.plotly_chart(fig9)

    if 'Timestamp' in claims.columns:
        claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce')
        claims_by_day = claims['Timestamp'].dt.date.value_counts().sort_index()
        fig10 = px.line(x=claims_by_day.index, y=claims_by_day.values,
                        labels={'x': 'Date', 'y': 'Number of Claims'},
                        title='Claims Over Time')
        st.plotly_chart(fig10)
    
    #------------------------------------------------------------------------------------------------------------


elif page == 'SQL Queries':
    st.title("🧮 SQL Queries Dashboard")

    queries = {
        "1. Food providers and receivers in each city": """
        SELECT
            p.City,
            COUNT(DISTINCT p.Provider_ID) AS Num_Providers,
            COUNT(DISTINCT r.Receiver_ID) AS Num_Receivers
        FROM providers p
        LEFT JOIN receivers r ON p.City=r.City
        GROUP BY p.City
        """,

        "2. Food provider type contributing most": """
        SELECT 
            p.Type,
            SUM(f.Quantity) AS Food_Total
        FROM foodlistings f
        JOIN providers p on f.Provider_ID=p.Provider_ID
        GROUP BY p.Type
        ORDER BY Food_Total DESC
        LIMIT 1;
        """,

        "3. Contact info of providers in city": f"""
        SELECT DISTINCT Name,Contact
        FROM providers p
        WHERE City=%s;
        """,

        "4. Receivers who claimed most food": """
        SELECT r.Name, SUM(f.Quantity) AS Food_Claimed
        FROM claims c
        JOIN receivers r ON c.Receiver_ID =r.receiver_ID
        JOIN foodlistings f ON c.Food_ID=f.Food_ID
        WHERE c.Status='Completed' 
        GROUP BY r.Name
        ORDER BY Food_Claimed DESC
        LIMIT 1;
        """,

        "5. Total quantity of food available": """
        SELECT SUM(f.quantity) as Total_Food FROM foodlistings f
        """,

        "6. City with highest food listings": """
        SELECT Location AS City, COUNT(*) AS Max_Listings
        FROM foodlistings
        GROUP BY Location
        ORDER BY Max_Listings DESC
        LIMIT 1;
        """,

        "7. Most commonly available food types": """
        SELECT f.Food_Type, COUNT(*) AS Common_Type
        FROM foodlistings f
        GROUP BY f.Food_Type
        ORDER BY Common_Type DESC
        LIMIT 1;
        """,

        "8. Number of food claims per item": """
        SELECT c.Food_ID, COUNT(*) AS Num_Claims
        FROM claims c
        GROUP BY Food_ID
        ORDER BY Num_Claims DESC;
        """,

        "9. Provider with most successful claims": """
        SELECT p.Provider_ID, COUNT(*) AS Succesful_Claims
        FROM claims c
        JOIN foodlistings f ON c.Food_ID=f.Food_ID
        JOIN providers p ON p.Provider_ID= f.Provider_ID
        WHERE c.Status='Completed'
        GROUP BY p.Provider_ID
        ORDER BY Succesful_Claims DESC
        LIMIT 1;
        """,

        "10. Percentage of claim statuses": """
        SELECT 
                Status,
                COUNT(*) AS count,
                ROUND(100.0*COUNT(*)/(SELECT COUNT(*) FROM claims),2) AS Percentage
        FROM claims
        GROUP BY Status
        ORDER BY Percentage DESC;
        """,

        "11. Avg food claimed per receiver": """
        SELECT r.Name, AVG(f.Quantity) AS Avg_Claimed
        FROM claims c
        JOIN receivers r ON c.receiver_ID=r.receiver_ID
        JOIN foodlistings f ON c.Food_ID=f.Food_ID
        GROUP BY r.Name
        """,

        "12. Most claimed meal type": """
        SELECT Meal_Type, COUNT(*) AS Num_claims
        FROM foodlistings f
        JOIN claims c ON f.Food_ID=c.Food_ID
        GROUP BY Meal_type
        ORDER BY Num_claims DESC
        LIMIT 1;
        """,

        "13. Total food donated per provider": """
        SELECT p.Provider_ID, COUNT(*) AS Num_Donated
        FROM providers p
        JOIN foodlistings f ON p.Provider_ID=f.Provider_ID
        GROUP BY Provider_ID
        ORDER BY Num_Donated;
        """,

        "14. Avg claims per receiver": """
        SELECT Receiver_ID, COUNT(*) AS total_claims
        FROM claims
        GROUP BY Receiver_ID
        """,

        "15. Providers with most diverse offerings": """
        SELECT 
            p.Name AS Pname,
            COUNT(DISTINCT f.Food_type) AS Food_Type_Count
        FROM foodlistings f
        JOIN providers p ON p.Provider_ID =f.Provider_ID
        GROUP BY p.Name
        ORDER BY Food_Type_Count DESC
        LIMIT 1;
        """,

        "16. Top 5 active providers": """
        SELECT 
            p.Provider_ID,
            p.Name as Provider_name,
            COUNT(c.Claim_ID)  AS Claim_num
        FROM providers p
        JOIN foodlistings f ON p.Provider_ID=f.Provider_ID
        JOIN claims c ON f.Food_ID=c.Food_ID
        WHERE c.Status='Completed'
        GROUP BY p.Provider_ID,p.Name
        ORDER BY Claim_num DESC
        LIMIT 5; 
        """,

        "17. Distribution of claim status": """
        SELECT 
            Status,
            COUNT(Claim_ID) AS Claim_Count,
            ROUND(COUNT(Claim_ID)/(SELECT COUNT(*) FROM claims)*100,2) AS Claim_Percentage
        FROM claims 
        GROUP BY Status
        """,

        "18. Providers with no claims": """
        SELECT 
            p.Provider_ID,
            p.Name AS Provider_Name
        FROM providers p
        WHERE NOT EXISTS (
            SELECT 1
            FROM foodlistings f
            JOIN claims c ON f.Food_ID=c.Food_ID
            WHERE f.Provider_ID=p.provider_ID
        );
        """,

        "19. Total claims & quantity per meal": """
        SELECT 
            f.Meal_Type,
            COUNT(c.Claim_ID) AS Total_Claims,
            SUM(f.Quantity) AS Total_Quantity
        FROM claims c
        JOIN foodlistings f ON c.Food_ID=f.Food_ID
        GROUP BY f.Meal_Type, f.Quantity
        ORDER BY Total_Quantity DESC
        """,

        "20. Top 10 provider cities": """
        SELECT 
            p.City AS Provider_city,
            COUNT(*) AS Provider_by_city
        FROM providers p
        GROUP BY p.City
        ORDER BY Provider_by_city DESC
        LIMIT 10
        """,

        "21. Most common receiver type": """
        SELECT 
            r.Type as Receiver_type,
            COUNT(*) AS Receiver_type_count
        FROM receivers r
        GROUP BY r.Type
        ORDER BY Receiver_type_count DESC
        LIMIT 1;
        """,

        "22. Receiver type by count": """
        SELECT 
            r.Type AS Receiver_type,
            COUNT(*) AS Receiver_Type_Count
        FROM receivers r
        GROUP BY r.Type
        ORDER BY Receiver_Type_Count;
        """,

        "23. Receiver count by city": """
        SELECT  
            City AS Receiver_City,
            COUNT(*) AS Receiver_city_count
        FROM receivers
        GROUP BY City
        ORDER BY Receiver_city_count DESC
        """,

        "24. Providers with most expired food": """
        SELECT 
            p.Name AS Provider_Name,
            COUNT(f.Food_ID) AS Expired_food_count
        FROM foodlistings f
        JOIN providers p ON f.Provider_ID=p.Provider_ID
        WHERE f.Expiry_Date<CURDATE()
        GROUP BY p.Provider_ID, p.Name
        ORDER BY Expired_food_count DESC
        LIMIT 5;
        """
    }

    query_choice = st.selectbox("Select SQL Query", list(queries.keys()))

    if "Contact info" in query_choice:
        city_input = st.text_input("Enter City Name")
        if city_input:
            cursor.execute(queries[query_choice], (city_input,))
            df = pd.DataFrame(cursor.fetchall(), columns=["Name", "Contact"])
            st.dataframe(df)
    else:
        cursor.execute(queries[query_choice])
        rows = cursor.fetchall()
        if rows:
            columns = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(rows, columns=columns)
            st.dataframe(df)
        else:
            st.write("No data found for the selected query.")

elif page == 'Writer Info':
    #st.title("Sadguru Kripa")
    st.title("🧑‍💻 Developer Information")
    st.markdown("""
    ### 📌 Project Creator  
    **Name:** Ananya R  
    **Project:** Local Food Waste Management System  
    **Role:** Design & Development  
    """)
    
    st.markdown("""
    ### 🔗 Connect  
    - [LinkedIn Profile](https://www.linkedin.com/in/ananyar-80595a269/)
    """)
    st.markdown("---")

    st.markdown("""
    _"This project was built with the aim of optimizing modern data-driven techniques to facilitate quick and efficient management of real-world problems around food waste management."_  
    — *Ananya R*
    """)



else:
    print("Incorrect value error")




Overwriting food.py


In [82]:
! streamlit run food.py

^C


In [ ]:
#Sadgurucharanaravindarpanamastu